In [1]:
!pip install faiss-cpu langchain sentence_transformers rank-bm25
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
import os
import glob

def split_text_into_chunks(
    text: str,
    chunk_size: int = 512,
    overlap: int = 100
) -> list:
    """
    text를 chunk_size씩 잘라서 리스트로 반환.
    각 청크 사이에 overlap 길이만큼 문자가 겹치도록 함.

    예) chunk_size=512, overlap=100 => 실제 step=412
    즉,
      - 첫 청크: [0:512]
      - 두 번째 청크: [412:412+512] (앞 100자가 겹침)
      - 세 번째 청크: [824:824+512]
      ...
    """
    chunks = []
    step = chunk_size - overlap
    if step <= 0:
        raise ValueError("chunk_size must be greater than overlap")

    text_length = len(text)
    start = 0

    while start < text_length:
        end = min(start + chunk_size, text_length)
        chunk = text[start:end]
        chunks.append(chunk)

        start += step  # 겹치기 고려하여 다음 청크 시작점 이동

    return chunks


def load_docs_from_txt(directory: str, chunk_size=1024, overlap=200):
    """
    지정한 디렉토리에 있는 모든 txt 파일을 읽은 후,
    chunk_size, overlap으로 청크를 분할하여
    [{'doc_id': ..., 'file_name': ..., 'chunk_index': ..., 'chunk_text': ...}, ...] 형태 리스트를 반환
    """
    docs = []
    doc_id = 0

    for file_path in glob.glob(os.path.join(directory, '*.txt')):
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read().strip()

        chunks = split_text_into_chunks(text, chunk_size, overlap)

        for i, chunk_text in enumerate(chunks):
            docs.append({
                "doc_id": doc_id,
                "file_name": os.path.basename(file_path),
                "chunk_index": i,
                "chunk_text": chunk_text
            })
        doc_id += 1

    return docs

# load_docs_from_txt('/content/drive/MyDrive/BITAmin/컨퍼런스/data')


In [3]:
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

def build_faiss_index(docs):
    """
    주어진 문서(docs)를 LangChain Document로 변환 후
    Faiss 인덱스를 생성 & 반환
    """
    # 1) 임베딩 모델 로드 (E5 모델 예시)
    embedding_model = "intfloat/multilingual-e5-large"
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

    # 2) langchain Document로 변환
    #    => chunk_text를 page_content로, 나머지를 metadata로
    langchain_docs = []
    for d in docs:
        langchain_docs.append(Document(
            page_content=d["chunk_text"],
            metadata={
                "doc_id": d["doc_id"],
                "file_name": d["file_name"],
                "chunk_index": d["chunk_index"]
            }
        ))

    # 3) FAISS VectorStore 생성
    faiss_store = FAISS.from_documents(langchain_docs, embeddings)

    # (옵션) 로컬에 인덱스 저장
    # faiss_store.save_local("faiss_index")

    return faiss_store


In [4]:
from rank_bm25 import BM25Okapi

def build_bm25_index(docs):
    """
    docs 목록을 받아서 BM25 인덱스(BM25Okapi) 객체와
    문서 정보(문서 리스트)를 반환
    """
    # docs: [{'doc_id':..., 'chunk_text':...}, ...]

    # BM25Okapi에 넣을 때는 토큰화가 필요
    # 간단히 whitespace split(또는 konlpy, mecab 등 사용 가능)
    tokenized_corpus = []
    for d in docs:
        tokens = d["chunk_text"].split()
        tokenized_corpus.append(tokens)

    bm25 = BM25Okapi(tokenized_corpus)
    return bm25


In [6]:
def main():
    docs_directory = "/content/drive/MyDrive/BITAmin/컨퍼런스/data"
    docs = load_docs_from_txt(docs_directory, chunk_size=2024, overlap=300)
    print(f"Loaded {len(docs)} docs (chunked)")

    # 1) Build Faiss (dense)
    faiss_store = build_faiss_index(docs)
    print("[INFO] Faiss index built.")

    # (옵션) 저장
    faiss_store.save_local("faiss_index")

    # 2) Build BM25 (sparse)
    bm25 = build_bm25_index(docs)
    print("[INFO] BM25 index built.")

if __name__ == "__main__":
  main()

Loaded 232 docs (chunked)


<ipython-input-3-076dbb44c79b>:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

[INFO] Faiss index built.
[INFO] BM25 index built.


In [8]:
import numpy as np
from heapq import nlargest

def hybrid_search(
    query: str,
    faiss_store: FAISS,
    bm25,
    docs,
    embeddings: HuggingFaceEmbeddings,
    alpha: float = 0.5,
    top_k: int = 5
):
    """
    query에 대해:
      1) Faiss에서 top-N (~= top_k * 5 정도) 문서를 찾고, dense 점수(= 코사인 유사도) 계산
      2) BM25 점수(스파스)를 docs 전체에 대해 계산
      3) 두 점수를 가중합(alpha)으로 결합해 상위 top_k 문서를 최종 선정

    - alpha: dense 점수와 BM25 점수를 어떻게 섞을지 결정 (0.0 ~ 1.0)
    - top_k: 최종적으로 반환할 문서 개수
    """

    # 1) query 임베딩 구하기
    query_embedding = embeddings.embed_query(query)
    query_embedding = np.array(query_embedding, dtype=np.float32)

    # 2) Faiss 쪽 검색
    #  - langchain >= 0.0.164 버전에는 `similarity_search_with_score()`
    #    만약 지원 안 하면, 일단 top-N개의 문서를 가져와서 재계산.
    #    아래 예시로 top 50개 문서를 대상으로 하이브리드 처리
    N = max(top_k*5, 50)

    dense_results = faiss_store.similarity_search_with_score(query, k=N)
    # dense_results: List[ (Document, float(similarity)) ]
    # similarity: cos 유사도 (1.0에 가까울수록 유사) - 버전에 따라 다를 수 있음

    # 3) BM25 쪽 검색
    #    - docs 전체에 대해 query의 BM25 점수를 구한다
    from rank_bm25 import BM25Okapi
    query_tokens = query.split()
    bm25_scores = bm25.get_scores(query_tokens)  # length = len(docs)

    # 4) dense_results를 doc_id 기준으로 매핑 { index_in_docs : dense_score }
    doc2dense = {}
    # docs 리스트와 dense_results가 "어떤 index"로 연결되어 있는지 알아야 함
    # 보통 doc.metadata["doc_id"], doc.metadata["chunk_index"] 등을 사용
    for (doc_obj, score) in dense_results:
        md = doc_obj.metadata
        d_id = md["doc_id"]
        c_idx = md["chunk_index"]
        # docs 리스트에서 (doc_id, chunk_index)가 어느 인덱스인지 찾는 로직 필요
        # 예: "key = (doc_id, chunk_index)"
        # 이걸 docs에서 검색하거나, 미리 build 해둔 인덱스(lookup)로 찾을 수 있음
        # 아래는 간단히 docs를 loop 돌며 찾을 수도 있음(비효율적).
        # -> 실제로는 dict[(doc_id, chunk_index)] = i 식으로 미리 만들어두면 좋음

        # 간단히 "d_id * 1_000_000 + c_idx"를 key로:
        unique_key = d_id * 1_000_000 + c_idx
        doc2dense[unique_key] = score

    # 5) 최종 점수를 합산(hybrid)하기
    #   - docs[i]에 대해서:
    #       BM25 점수 = bm25_scores[i]
    #       Dense 점수 = doc2dense.get(unique_key, 0)
    #     => final_score = alpha * dense + (1-alpha) * bm25
    #   - BM25와 Dense 스케일이 다를 수 있으므로, 실제로는 min-max 정규화나,
    #     z-score 표준화가 필요할 수도 있음 (여기서는 단순 예시)

    results = []
    for i, d in enumerate(docs):
        d_id = d["doc_id"]
        c_idx = d["chunk_index"]
        unique_key = d_id * 1_000_000 + c_idx

        dense_s = doc2dense.get(unique_key, 0.0)      # 없으면 0
        sparse_s = bm25_scores[i]                    # BM25 점수
        final_s = alpha*dense_s + (1-alpha)*sparse_s

        results.append((final_s, i))

    # 6) 최종 상위 top_k 선별
    results.sort(key=lambda x: x[0], reverse=True)
    top_indices = [idx for (_, idx) in results[:top_k]]

    # 7) 반환용
    out = []
    for ti in top_indices:
        out.append(docs[ti])  # docs[ti] 구조: {'doc_id','chunk_index','chunk_text','file_name',...}
    return out


In [18]:
def main():
    docs_directory = "/content/drive/MyDrive/BITAmin/컨퍼런스/data"
    docs = load_docs_from_txt(docs_directory, chunk_size=2024, overlap=300)

    # FAISS 인덱스
    faiss_store = build_faiss_index(docs)
    # BM25 인덱스
    bm25 = build_bm25_index(docs)

    # 같은 임베딩 객체를 하이브리드로도 사용
    embedding_model = "intfloat/multilingual-e5-large"
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

    query = "전봉준"   # 예시 질문
    results = hybrid_search(query, faiss_store, bm25, docs, embeddings, alpha=0.5, top_k=3)

    print(results)

if __name__ == "__main__":
  main()


[{'doc_id': 8, 'file_name': 'ocr_result_chunk_10.txt', 'chunk_index': 5, 'chunk_text': '후가 말년에 병으로 자주 피접을 갔는데, 마지막으로 요양한 곳이 바로 둘째 아들인 세조의 개인 사저였을 정도지요. 또한 세조는 보기와 달리 무척 애처가였답니다. 가계도를 살펴볼까요? \n제7대 세조에게는 정희왕후 윤씨와 그 아래 덕종(의경세자), 제 8대 예종이 있었고, 후궁 근빈 박씨가 있었어요. \n세조에게는 후궁이 딱한 명밖에 없다는 걸 알 수 있지요? 아버지 세종과 할아버지 태종은 후궁도 많고 자식도 많았는데도 말이지요. 하지만 세조는 아버지, 할아버지와는 달랐습니다. \n당시 세조는 아내였던 정희왕후를 매우 사랑했다고 해요. 조선시대에는 왕이 오직 한 명의 부인하고만 백년해로 하고 싶어도, 신하들이 가만 놔두질 않아요. 후사를 생산해야 하니까요. 그래서 어쩔 수 없이 후궁을 들여야 했지요. 세조 역시 신하들의 등쌀에 못 이겨 후궁을 들이는데, 딱 한 명만 들입니다. 세조는 나들이를 나갈 때는 물론 이거니와 심지어 사냥터에 갈 때도, 부인을 대동했을 만큼 부부 금슬이 좋았다고 해요. \n\n"임금이 중궁(中홈)과 더불어 등교(후회)에 거둥하여 사냥하는 것을 구경하였다." \n「세조실록」 27권, 8년(1462) 2월 27일 \n\n심지어 신하들과 토론을 하는 자리에서도 "우리 집사람이 말하기를"이란 말을 버릇처럼 입에 달고 살았을 만큼 소문난 애처가였다고 하네요. \n\n【 애주가 세조 ] \n세조는 자주 신하들과 술자리를 가졌답니다. 어느 날 자신을 왕으로 만들어준 한명회, 신숙주와 함께 술을 마셨습니다. 신숙주는 원래 집현전 학자였지만 세조 쪽으로 돌아선 변절의 아이콘이에요. 술을 마시던 도중, 세조와 신숙주가 팔씨름을 하지요. 감히 왕하고 팔씨름해서 이기려고 드는 것 자체가 어리석은 짓이지요. 그런데 이게 웬일인가요, 신숙주가 세조의 팔을 한방에 확 꺾어버려요. \n윽! \n세조의 외마디가 울

In [16]:
results

NameError: name 'results' is not defined